File content:

Touluse CSV creation: Code for correct the street name and create the CSV files.

Toulouse database and SQL queries.

TOULOUSE - FRANCE IN OSM.
Postal codes 31000, 31100, 31200,31300, 31400 and 31500 + CEDEX (those are the postal codes of Toulouse urban area).
The Coordinates we will extract are: 43.6552N, 1.5024E, 43.5595S, 1.3935W

In [1]:
# This is the list of imports we are going to use

import xml.etree.ElementTree as ET
from collections import defaultdict
import re
import pprint
import os
import os.path
import sys
import time
import requests
import codecs
import json
from IPython.display import Image
import csv
import codecs
import sqlite3


We define the corrections to be pass when CSV creation

In [13]:
#Function to correct street name when creating CSV files

dataset = 'UdcT2'

street_type_re = re.compile(r'(?P<word>)(\b\w+\b)', re.IGNORECASE)

street_types = defaultdict(int)

#List of expected street types

expected = ['Rue', 'Allée', 'Avenue',  'Place', 'Impasse', 'Route', 'Chemin', 'Boulevard', 
            'Allées', 'Esplanade', 'Port', 'Promenade', 'Quai', 'Passage', 'Cheminement', 
            'Voie', 'Descente', 'Square', 'Contre-Allée','Périphérique', 'Cours', 'Parvis', 
            'Grande','Angle','Sur','face']

#Dictionary to correct the capitals missing in street type
cap_mapping = { 'route' :'Route',
            'ROUTE' :'Route',
            'rue' : 'Rue' ,
            'AVENUE': 'Avenue',
            'avenue': 'Avenue',
            'place':  'Place' ,
            'allées' :'Allées',
            'allée': 'Allée',
            'voie' :'Voie',
           'chemin':'Chemin'
                }

# Dictionary to correct the wrong naming of street
other_mapping = {'107':'Cours Rosalind Franklin, 107',
         '6' :'Impasse Leonce Couture, 6' ,
           '9': 'Rue Reclusane, 9'  ,
          'Frédéric':'Rue Frédéric Petit',
          'Lotissement':'Impasse René Mouchotte_Lotissement Futuropolis'
               }

# Fucntion to pring a dict sorted by key
def print_sorted_dict(d, expression):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print (expression % (k, v))

# Function to find the street in the file
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

# Function to find the street with type not in expected type
def expected_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


# Function to find the street with not expected street type in the file
def audit_st_tp(filename):
    problem_street_types = defaultdict(set)
    for event, elem in ET.iterparse(filename):
        if is_street_name(elem):
            expected_street_type(problem_street_types, elem.attrib['v'])
    return problem_street_types

#Function to define the correct streets name with wrong naming
def other_correct(street_name, street_type):
    if type(other_mapping[street_type]) == type('string'):
        name = other_mapping[street_type]
    else:
        for key in other_mapping[street_type]:
                name = key
    return name

# Function to correct the names of the street        
def update_name(name):
    street_type= name.split(' ',1)[0]
    street_name= name.split(' ',1)[-1]        

    if street_type in cap_mapping:
        name = cap_mapping[street_type] + ' ' + street_name  
    elif street_type in other_mapping:
        name = other_correct(street_name, street_type)
    return name


#Function to create the dictionary that we will use to change the names of the streets when creating CSV
def run_updates(filename):
    st_types = audit_st_tp(dataset)
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name) # voy a prograr update name
            if better_name != name:
                corrected_names[name] = better_name
    return corrected_names

#Create the dictionary
corrected_names = {}   
corrected_names = run_updates(dataset)

#Print the correct names
print_sorted_dict(corrected_names, "%s: %s")  

107: Cours Rosalind Franklin, 107
6 Impasse Leonce Couture: Impasse Leonce Couture, 6
9: Rue Reclusane, 9
allée Fernand Jourdant: Allée Fernand Jourdant
allées Jules-Guesde: Allées Jules-Guesde
avenue de Lombez: Avenue de Lombez
avenue du Colonel Roche: Avenue du Colonel Roche
avenue Etienne Billieres: Avenue Etienne Billieres
AVENUE JEAN RIEUX: Avenue JEAN RIEUX
avenue jean-chaubet: Avenue jean-chaubet
avenue Jean-Rieux: Avenue Jean-Rieux
chemin Cassaing: Chemin Cassaing
chemin de la Salade Ponsan: Chemin de la Salade Ponsan
Frédéric Petit: Rue Frédéric Petit
Lotissement Futuropolis - Impasse René Mouchotte: Impasse René Mouchotte_Lotissement Futuropolis
place Papus: Place Papus
place Saint-Cyprien: Place Saint-Cyprien
ROUTE DE BLAGNAC: Route DE BLAGNAC
route de Fronton: Route de Fronton
route de launaguet: Route de launaguet
rue Baour-Lormian: Rue Baour-Lormian
rue Bénezet: Rue Bénezet
rue de Cugnaux: Rue de Cugnaux
rue de l'ayga: Rue de l'ayga
rue de l'industrie: Rue de l'industrie


In [14]:
# This is the schema for the CSV creation

SCHEMA = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


In [15]:
# Creation of CSV 

dataset = 'UdcT2'

NODES_PATH = "nodesT.csv"
NODE_TAGS_PATH = "nodes_tagsT.csv"
WAYS_PATH = "waysT.csv"
WAY_NODES_PATH = "ways_nodesT.csv"
WAY_TAGS_PATH = "ways_tagsT.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# Funtion to correct the street
def correct_element(v):
    if v in corrected_names:
        correct_value = corrected_names[v]
    else:
        correct_value = v
    return correct_value

#Funntion to define nodes and ways
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['user'] = element.attrib['user']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']
        
        for node in element:
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if ':' in node.attrib['k']:
                tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                tag_dict['value'] = correct_element(node.attrib['v'])
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = node.attrib['k']
                tag_dict['value'] = correct_element(node.attrib['v'])
            tags.append(tag_dict)
            
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['version'] = element.attrib['version']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']
        n = 0
        for node in element:
            if node.tag == 'nd':
                way_dict = {}
                way_dict['id'] = element.attrib['id']
                way_dict['node_id'] = node.attrib['ref']
                way_dict['position'] = n
                n += 1
                way_nodes.append(way_dict)
            if node.tag == 'tag':
                tag_dict = {}
                tag_dict['id'] = element.attrib['id']
                if ':' in node.attrib['k']:
                    tag_dict['type'] = node.attrib['k'].split(':', 1)[0]
                    tag_dict['key'] = node.attrib['k'].split(':', 1)[-1]
                    tag_dict['value'] = correct_element(node.attrib['v'])
                else:
                    tag_dict['type'] = 'regular'
                    tag_dict['key'] = node.attrib['k']
                    tag_dict['value'] = correct_element(node.attrib['v'])
                tags.append(tag_dict)
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })
    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w',encoding="utf-8") as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'w',encoding='utf-8') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w',encoding='utf-8') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w',encoding='utf-8') as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w',encoding='utf-8') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow((el['way']))
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
process_map(dataset)

We have now the CSV file created, we can create our database

In [5]:
# Creating database on disk

sqlite_file = 'UdcTls.db'

conn = sqlite3.connect(sqlite_file)

c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS nodes''')
c.execute('''DROP TABLE IF EXISTS nodes_tags''')
c.execute('''DROP TABLE IF EXISTS ways''')
c.execute('''DROP TABLE IF EXISTS ways_tags''')
c.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()



QUERY_NODES = """
CREATE TABLE nodes (
    id INTEGER NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
"""

QUERY_NODES_TAGS = """
CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);
"""

QUERY_WAYS = """
CREATE TABLE ways (
    id INTEGER NOT NULL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);
"""

QUERY_WAYS_TAGS = """
CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);
"""

QUERY_WAYS_NODES = """
CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
"""



c.execute(QUERY_NODES)
c.execute(QUERY_NODES_TAGS)
c.execute(QUERY_WAYS)
c.execute(QUERY_WAYS_TAGS)
c.execute(QUERY_WAYS_NODES)

conn.commit()

In [6]:
#Creating structure of database

with open('nodes.csv','rt', encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db1 = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('nodes_tags.csv','rt',encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db2 = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    
with open('ways.csv','rt', encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db3 = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
with open('ways_tags.csv','rt', encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db4 = [(i['id'], i['key'], i['value'], i['type']) for i in dr]
    
with open('ways_nodes.csv','rt', encoding='utf8') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db5 = [(i['id'], i['node_id'], i['position']) for i in dr]


In [7]:
#Filling the database

c.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db1)
c.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db2)
c.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db3)
c.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db4)
c.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db5)
conn.commit()

In [12]:
# Query to count the nodes and ways in database
c.execute('SELECT COUNT(*) FROM nodes')
all_rows = c.fetchall()
print('Nodes in the data base:', all_rows)

c.execute('SELECT COUNT(*) FROM ways')
all_rows = c.fetchall()
print('Ways in data base:',all_rows)

Nodes in the data base: [(956186,)]
Ways in data base: [(160279,)]


In [39]:
# Query to show the nicknames *user* and contributions of the top 15 contributors
QUERY = '''
SELECT DISTINCT nodes.user, COUNT(*)
FROM nodes
GROUP BY nodes.user
ORDER BY COUNT(*) DESC
LIMIT 15;
'''
c.execute(QUERY)
all_rows = c.fetchall()
print('Top 15 contributors and their contributions:',all_rows)

Top 15 contributors and their contributions: [('Pinpin', 296412), ('emarsden', 189554), ('don-vip', 69909), ('Sebastien Dinot', 60055), ('Hervé TUC', 47052), ('Tyndare', 27593), ('isnogoud', 26753), ('square', 24542), ('Florence Birée', 21728), ('capdarmen', 21454), ('FredB', 20517), ('orhygine', 15530), ('Luuuddooo', 12966), ('Carboon33', 10143), ('Floeditor', 7126)]


In [40]:
# Query to show nicknames *user* and contributions of the top 10 contributors in %
QUERY = '''
SELECT DISTINCT nodes.user, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes)
FROM nodes
GROUP BY nodes.uid
ORDER BY (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes)) DESC
LIMIT 10;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Top 10 contributors and the % of their contributions:', all_rows)

Top 15 contributors and their contributions: [('Pinpin', 30.99940806495807), ('emarsden', 19.823967303432596), ('don-vip', 7.311234425101392), ('Sebastien Dinot', 6.280681792036278), ('Hervé TUC', 4.920799928047472), ('Tyndare', 2.885735620475514), ('isnogoud', 2.7978866036524273), ('square', 2.566655441514517), ('Florence Birée', 2.2723612351571765), ('capdarmen', 2.243705722526789)]


In [48]:
# Query to see the source of information
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='source'
GROUP BY value
ORDER BY Count DESC
LIMIT 15;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Top 15 source of informations:',all_rows)

Top 15 source of informations: [('ToulouseMetropole', 35380), ('GrandToulouse', 16958), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2009', 8113), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2011', 3445), ('survey', 1924), ('Bing', 1442), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2010', 974), ('http://data.grandtoulouse.fr', 718), ('https://www.tisseo.fr', 624), ('ToulouseMetropole - Orthophotoplan 2015', 611), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre ; mise à jour : 2008', 571), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2014', 558), ('Enedis - 06/2018', 449), ('INSEE - 06/2019', 288), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre ; mise à jour : 2011', 195)]


In [43]:
# Query to show % of information by source 
QUERY = '''
SELECT value, COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes_tags)
FROM nodes_tags
WHERE key='source'
GROUP BY value
ORDER BY (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM nodes)) DESC
LIMIT 10;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Top 10 source of informations in %:',all_rows)

Top 10 source of informations in %: [('ToulouseMetropole', 11.459888769762477), ('GrandToulouse', 5.492843237920636), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2009', 2.6278710454800165), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2011', 1.1158653706001056), ('survey', 0.6232002824483609), ('Bing', 0.4670763031655594), ('cadastre-dgi-fr source : Direction Générale des Impôts - Cadastre. Mise à jour : 2010', 0.3154870452727149), ('http://data.grandtoulouse.fr', 0.2325664255706461), ('https://www.tisseo.fr', 0.20211901052379272), ('ToulouseMetropole - Orthophotoplan 2015', 0.19790819780454702)]


In [24]:
# Query to count the tags with more nodes, show top 10
QUERY = '''
SELECT ways_tags.value, COUNT(*)
FROM ways_tags
WHERE ways_tags.key = 'name'
AND ways_tags.type = 'regular'
GROUP BY ways_tags.value
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print(all_rows)

[('Avenue des États-Unis', 70), ('Boulevard André Netwiller', 65), ('Route de Narbonne', 65), ("Avenue d'Atlanta", 64), ("Route d'Albi", 55), ('Ligne de Bordeaux à Sète', 53), ('Ligne de Toulouse à Bayonne', 53), ('Rue Saint-Jean', 49), ('Autoroute des Deux Mers', 48), ('Allées Charles de Fitte', 46)]


In [15]:
# Query to count the top 15 amenities
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY Count DESC
LIMIT 15;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print(all_rows)

[('bench', 985), ('recycling', 895), ('restaurant', 726), ('bicycle_parking', 667), ('waste_basket', 443), ('fast_food', 356), ('bicycle_rental', 286), ('post_box', 279), ('parking', 275), ('vending_machine', 187), ('drinking_water', 181), ('bar', 178), ('cafe', 172), ('pharmacy', 163), ('bank', 157)]


If after reading this project you decide to come to visit Touluse you will have enouth place to eat with 726 restaurant, 356 fast-food, 178 bar and 172 cafe
And if you need money you can go to one of the 157 banks
You can rent a bicycle in one of the 286 rental point
Or send a postcard to your friends from one of the 279 post box
If after all that you are tyred you can set in one fo the 985 bench that are spread in the city

In [28]:
# Query to count the cuisine of the restaurant
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as count
FROM nodes_tags 
JOIN
    (SELECT DISTINCT(id)
    FROM nodes_tags
    WHERE value='restaurant') as Sub
ON nodes_tags.id=Sub.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY Count DESC;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print(all_rows)

[('french', 91), ('pizza', 42), ('regional', 29), ('japanese', 26), ('asian', 25), ('indian', 21), ('italian', 20), ('chinese', 17), ('vietnamese', 13), ('burger', 11), ('lebanese', 9), ('pancake', 7), ('seafood', 6), ('thai', 6), ('crepe', 5), ('spanish', 5), ('sushi', 5), ('african', 4), ('moroccan', 4), ('brasserie', 3), ('couscous', 3), ('international', 3), ('kebab', 3), ('tapas', 3), ('american', 2), ('arab', 2), ('argentinian', 2), ('caribbean', 2), ('latin_american', 2), ('local', 2), ('mexican', 2), ('sandwich', 2), ('steak_house', 2), ('Gastronomique', 1), ('american;burger;brunch', 1), ('asian;thai', 1), ('asian;vietnamese', 1), ('asian;wok;japanese;chinese;regional', 1), ('beer;bistro', 1), ('bio', 1), ('burger;friture;belgian;mussel', 1), ('burger;pizza;french', 1), ('burger;tapas', 1), ('burger;tapas;french', 1), ('cake', 1), ('cake;teahouse', 1), ('chicken', 1), ('chilean', 1), ('corsican', 1), ('coréen;Asiatique;Végétarien;Végétalien;sandwich;japanese;beef_bowl;Poke_Bow

When we check the type of cuisien we have a winner: the French cuisine, what such surprise!!!
But if you finaly come, I recomend you to taste the local cuisine in one of the 29 regional restaurant

In [73]:
# Query to see the brands of fast-food 
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as count
FROM nodes_tags 
JOIN
    (SELECT DISTINCT(id)
    FROM nodes_tags
    WHERE value='fast_food') as Sub
ON nodes_tags.id=Sub.id
WHERE nodes_tags.key='brand'
GROUP BY nodes_tags.value
ORDER BY Count DESC;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Do yo prefer fast-food, we have some too:',all_rows)

Do yo prefer fast-food, we have some too: [('Subway', 7), ("McDonald's", 6), ("Domino's", 4), ("Domino's Pizza", 3), ('Burger King', 2), ('KFC', 2), ("O'Tacos", 2), ('Quick', 2), ('Bagelstein', 1), ('Big Fernand', 1), ('Pizza Pizza', 1), ('Sushi Shop', 1), ('Tutti Pizza', 1)]


In [53]:
# Query to count the top 10 shop types
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='shop'
GROUP BY value
ORDER BY Count DESC
LIMIT 10;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Do you want to go shopping?',all_rows)

Do you want to go shopping? [('clothes', 466), ('hairdresser', 279), ('bakery', 181), ('convenience', 159), ('beauty', 111), ('jewelry', 110), ('butcher', 100), ('shoes', 94), ('vacant', 91), ('optician', 82)]


In [74]:
# Query to find the normal time the mail is colected in the post office
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='collection_times'
GROUP BY value
ORDER BY Count DESC
LIMIT 15;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('You want to know at what time the postcard for your friend will be collected:',all_rows)

You want to know at what time the postcard for your friend will be collected: [('Mo-Fr 12:00, Sa 11:00', 38), ('Mo-Fr 12:00; Sa 11:00', 4), ('Mo-Fr 16:00, Sa 11:00', 4), ('Mo-Fr 12:00, Sa 09:00', 3), ('Mo-Fr 08:30-18:30;Sa 09:00-12:30', 1), ('Mo-Fr 12:00, Sa unknown', 1), ('Mo-Fr 12:00,Sa 11:00', 1), ('Mo-Fr 15:00, Sa 11:00', 1), ('Mo-Fr 16:30; Sa 11:15', 1), ('Mo-Fr 17:00, Sa 12:00', 1), ('Mo-Fr 17:00; Sa 12:00', 1), ('Mo-Fr 18:00, Sa 11:00', 1), ('Mo-Sa 9:30', 1)]


In [68]:
# Query to find the type of sport places
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='sport'
GROUP BY value
ORDER BY Count DESC
LIMIT 15;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Now some sport:',all_rows)

Now some sport: [('fitness', 28), ('boules', 10), ('multi', 8), ('table_tennis', 7), ('climbing', 4), ('basketball', 3), ('boxing', 3), ('soccer', 3), ('swimming', 3), ('yoga', 3), ('aikido', 2), ('crossfit', 2), ('gymnastics', 2), ('handball', 2), ('rowing', 2)]


We have 4 places for climbing, as we need to be trainned for our trips to Pyrinees

In [71]:
# Query to find the tourism things 
QUERY = '''
SELECT value, COUNT(*) as Count
FROM nodes_tags
WHERE key='tourism'
GROUP BY value
ORDER BY Count DESC
LIMIT 15;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('But you come for tourism, dont you?:',all_rows)

But you come for tourism, dont you?: [('information', 142), ('artwork', 83), ('hotel', 65), ('gallery', 28), ('attraction', 24), ('picnic_site', 20), ('viewpoint', 12), ('museum', 10), ('apartment', 3), ('guest_house', 3), ('caravan_site', 1), ('gallerie', 1), ('hostel', 1), ('motel', 1), ('theme_park', 1)]


In [77]:
# Query to see the brands of fast-food 
QUERY = '''
SELECT nodes_tags.value, COUNT(*) as count
FROM nodes_tags 
JOIN
    (SELECT DISTINCT(id)
    FROM nodes_tags
    WHERE value='hotel') as Sub
ON nodes_tags.id=Sub.id
WHERE nodes_tags.key='name'
GROUP BY nodes_tags.value
ORDER BY Count DESC;
'''

c.execute(QUERY)
all_rows = c.fetchall()
print('Now you need to find a place for the night, you can chose amoung those:',all_rows)

Now you need to find a place for the night, you can chose amoung those: [('Ibis', 3), ('Novotel', 2), ('Adagio Access', 1), ('Adagio Access Toulouse St Cyprien', 1), ('Apart Hotel Citadines', 1), ('Aparthôtel Lagrange City Toulouse Saint-Michel', 1), ("Appart'City Hippodrome", 1), ('At Home Appart Hotel', 1), ('Auberge de Jeunesse', 1), ('B&b Hôtel', 1), ('B&b Hôtel Toulouse Centre', 1), ('Best Western Athénée', 1), ('Best Western Les Capitouls', 1), ('Castellane', 1), ('Crowne Plaza', 1), ('Excelsior', 1), ('Garden Hôtel', 1), ("Grand Hôtel D'Orléans", 1), ("Grand Hôtel de l'Opéra", 1), ('Hôtel Arnaud Bernard', 1), ('Hôtel Chartreuse', 1), ('Hôtel Croix Baragnon', 1), ('Hôtel Hermès', 1), ('Hôtel Icare', 1), ('Hôtel Junior', 1), ('Hôtel Kyriad', 1), ('Hôtel Le Prado', 1), ('Hôtel Mercure Toulouse Centre Compans', 1), ('Hôtel Mermoz', 1), ('Hôtel Occitania', 1), ('Hôtel Orsay', 1), ('Hôtel Raymond 4', 1), ('Hôtel ResidHome', 1), ('Hôtel Riquet', 1), ('Hôtel Saint-Sernin', 1), ('Hôtel V

And that is all about my city, hope you enjoy the trip.